<a href="https://colab.research.google.com/github/mushamvineeth/Vineeth_INFO5731_Spring2021/blob/main/INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [1]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [2]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [5]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [10]:
driver = webdriver.Chrome('chromedriver',options=options)
Review_Link = 'https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv'
Titles= []
Reviews= []
driver.get(Review_Link)
for p in range(4):
  driver.find_element_by_class_name("ipl-load-more__button").click()
  time.sleep(5)
  Complete_Titles = driver.find_elements(By.CLASS_NAME, "title")
  Complete_Reviews = driver.find_elements(By.CLASS_NAME, "text")
for m, n in zip(Complete_Reviews, Complete_Reviews):
      Titles.append((m.text).replace('\n',''))
      Reviews.append(n.text)
Data_Frame = pd.DataFrame(list(zip(Titles, Reviews)), columns =['Titles', 'Reviews'])
Data_Frame

,Titles,Reviews
0,"Every once in a while a movie comes, that trul...","Every once in a while a movie comes, that trul..."
1,This is a movie that only those who have felt ...,This is a movie that only those who have felt ...
2,"Truly a masterpiece, The Best Hollywood film o...","Truly a masterpiece, The Best Hollywood film o..."
3,Joaquin Phoenix gives a tour de force performa...,Joaquin Phoenix gives a tour de force performa...
4,Most of the time movies are anticipated like t...,Most of the time movies are anticipated like t...
...,...,...
120,I didnt really know much about this movie. I h...,I didnt really know much about this movie. I h...
121,,
122,Film was directed and produced poorly. Opportu...,Film was directed and produced poorly. Opportu...
123,"Wow, what a movie! I have to admit, When I fir...","Wow, what a movie! I have to admit, When I fir..."


In [14]:
#Lower Case
Data_Frame['After Converting to lower case'] = Data_Frame['Reviews'].apply(lambda x: " ".join(x.lower() for x in x.split()))
Data_Frame

#Punctuation Removal
Data_Frame['After Removing Punctuation'] = Data_Frame['After Converting to lower case'].str.replace('[^\w\s]','')
Data_Frame

#Special Charachters Removal
import re
Data_Frame['After Removing Special Charachters'] = Data_Frame['After Removing Punctuation'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
Data_Frame

#Stopwords Removal
from nltk.corpus import stopwords
stop = stopwords.words('english')
Data_Frame['After Removing Stopwords'] = Data_Frame['After Removing Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
Data_Frame

#Spelling Correction
from textblob import TextBlob
Data_Frame['After Spelling Correction'] = Data_Frame['After Removing Stopwords'].apply(lambda x: str(TextBlob(x).correct()))
Data_Frame

#Tokenization
import nltk
nltk.download('punkt')
Data_Frame['After Tokenization'] = Data_Frame['After Spelling Correction'].apply(lambda x: TextBlob(x).words)
Data_Frame


#Stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()
Data_Frame['After Stemming'] = Data_Frame['After Tokenization'].apply(lambda x: " ".join([ps.stem(word) for word in x]))
Data_Frame

#Lemmatization
from textblob import Word
import nltk
nltk.download('wordnet')
Data_Frame['After Lemmatization'] = Data_Frame['After Stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
Data_Frame.to_csv('/content/sample_data/reviwes.csv',index=False)
Data_Frame


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Titles,Reviews,After Converting to lower case,After Removing Punctuation,After Removing Special Charachters,After Removing Stopwords,After Spelling Correction,After Tokenization,After Stemming,After Lemmatization
0,"Every once in a while a movie comes, that trul...","Every once in a while a movie comes, that trul...","every once in a while a movie comes, that trul...",every once in a while a movie comes that truly...,every once in a while a movie comes that truly...,every movie comes truly makes impact joaquins ...,every movie comes truly makes impact joaquins ...,"[every, movie, comes, truly, makes, impact, jo...",everi movi come truli make impact joaquin perf...,everi movi come truli make impact joaquin perf...
1,This is a movie that only those who have felt ...,This is a movie that only those who have felt ...,this is a movie that only those who have felt ...,this is a movie that only those who have felt ...,this is a movie that only those who have felt ...,movie felt alone isolated truly relate underst...,movie felt alone isolated truly relate underst...,"[movie, felt, alone, isolated, truly, relate, ...",movi felt alon isol truli relat understand mot...,movi felt alon isol truli relat understand mot...
2,"Truly a masterpiece, The Best Hollywood film o...","Truly a masterpiece, The Best Hollywood film o...","truly a masterpiece, the best hollywood film o...",truly a masterpiece the best hollywood film of...,truly a masterpiece the best hollywood film of...,truly masterpiece best hollywood film 2019 one...,truly masterpiece best hollywood film 2019 one...,"[truly, masterpiece, best, hollywood, film, 20...",truli masterpiec best hollywood film 2019 one ...,truli masterpiec best hollywood film 2019 one ...
3,Joaquin Phoenix gives a tour de force performa...,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives tour de force performanc...,joaquin phoenix gives tour de force performanc...,"[joaquin, phoenix, gives, tour, de, force, per...",joaquin phoenix give tour de forc perform fear...,joaquin phoenix give tour de forc perform fear...
4,Most of the time movies are anticipated like t...,Most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,time movies anticipated like end falling short...,time moves anticipated like end falling short ...,"[time, moves, anticipated, like, end, falling,...",time move anticip like end fall short way shor...,time move anticip like end fall short way shor...
...,...,...,...,...,...,...,...,...,...,...
120,I didnt really know much about this movie. I h...,I didnt really know much about this movie. I h...,i didnt really know much about this movie. i h...,i didnt really know much about this movie i ha...,i didnt really know much about this movie i ha...,didnt really know much movie hadnt seen traile...,didn really know much movie hadn seen trailer ...,"[didn, really, know, much, movie, hadn, seen, ...",didn realli know much movi hadn seen trailer a...,didn realli know much movi hadn seen trailer a...
121,,,,,,,,[],,
122,Film was directed and produced poorly. Opportu...,Film was directed and produced poorly. Opportu...,film was directed and produced poorly. opportu...,film was directed and produced poorly opportun...,film was directed and produced poorly opportun...,film directed produced poorly opportunity take...,film directed produced poorly opportunity take...,"[film, directed, produced, poorly, opportunity...",film direct produc poorli opportun take advant...,film direct produc poorli opportun take advant...
123,"Wow, what a movie! I have to admit, When I fir...","Wow, what a movie! I have to admit, When I fir...","wow, what a movie! i have to admit, when i fir...",wow what a movie i hav

In [18]:
import itertools
import collections
from nltk import ngrams
from nltk.tokenize import word_tokenize

array= []
for j in Data_Frame['After Lemmatization']:
  array.append(word_tokenize(j))
cleaned_data = [x for x in array if x != []]
Repeat = list(itertools.chain.from_iterable(cleaned_data))

trigrams = nltk.trigrams(Repeat)
FD = nltk.FreqDist(trigrams)
FD

FreqDist({('everi', 'movi', 'come'): 1,
          ('movi', 'come', 'truli'): 1,
          ('come', 'truli', 'make'): 1,
          ('truli', 'make', 'impact'): 1,
          ('make', 'impact', 'joaquin'): 1,
          ('impact', 'joaquin', 'perform'): 1,
          ('joaquin', 'perform', 'scenographi'): 1,
          ('perform', 'scenographi', 'brillianc'): 1,
          ('scenographi', 'brillianc', 'grotesqu'): 1,
          ('brillianc', 'grotesqu', 'hunt'): 1,
          ('grotesqu', 'hunt', 'cring'): 1,
          ('hunt', 'cring', 'hard'): 1,
          ('cring', 'hard', 'watch'): 1,
          ('hard', 'watch', 'time'): 1,
          ('watch', 'time', 'mesmer'): 1,
          ('time', 'mesmer', 'wont'): 1,
          ('mesmer', 'wont', 'blink'): 1,
          ('wont', 'blink', 'eye'): 1,
          ('blink', 'eye', 'watch'): 1,
          ('eye', 'watch', 'tragic'): 1,
          ('watch', 'tragic', 'serious'): 1,
          ('tragic', 'serious', 'funni'): 1,
          ('serious', 'funni', 'moment

In [ ]:
from collections import Counter
BG = nltk.bigrams(Repeat)
FD = nltk.FreqDist(BG)
BG_dict = dict(FD)
for i in BG_dict:
  print( str(i) + ':' + str(BG_dict[i] / Repeat.count(i[0])))

In [50]:
complete_data = ''
j= []
count = 1
for i in Data_Frame['After Lemmatization']:
  complete_data = complete_data + i
  k = 'Review-' + str(i)
  j.append(k)
  count+= 1

from textblob import TextBlob
nph = []
fcy = []
for i in Data_Frame['After Lemmatization']:
  blob = TextBlob(i)
  for nouns in blob.noun_phrases:
    np.append(nouns)
for w in np:
  npf = []
  for i in Data_Frame['After Lemmatization']:
    npf.append(i.count(w) / complete_data.count(w))
  fcy.append(np)
noun_phrases_Data = pd.DataFrame(fcy).T
noun_phrases_Data.columns = list(np)
noun_phrases_Data.j = j
noun_phrases_Data

TypeError: ignored

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [39]:
# Write your code here
import math
def value_tf(s,w):
  length = len(s.split(" "))
  value_of_TF = s.count(w)/length
  value_of_IDF = 0
  if(value_of_TF!=0):
    value_of_IDF = math.log(length)/s.count(w)
  else:
    return 0;
  return value_of_TF*value_of_IDF

sentence = Data_Frame["After Spelling Correction"].values.tolist()
tl = set([j for i in sent for j in i.split(" ")])
IDF_TF = pd.DataFrame(tl,columns=["token"])
value = 0
for i in sentence:
  IDF_TF[str(count)] = IDF_TF["token"].apply(lambda x : value_tf(i,x))
  value=value+1

IDF_TF






,token,126
0,,0.21799
1,boring,0.00000
2,expel,0.00000
3,deserving,0.00000
4,trollworthy,0.00000
...,...,...
2275,notably,0.00000
2276,reviews,0.00000
2277,finally,0.00000
2278,reaction,0.00000


In [47]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

search_sentence="This is a movie that only those who have felt alone and isolated can truly relate to it. You understand the motive and you feel sorry for the character. A lot of people will see this movie and think that it encourages violence. But truly, this movie should encourage each and every one of us to become a better person, treat everyone with respect and make each other feel like they belong in this world, instead of making them feel isolated. ."

X_list = word_tokenize(search_sentence)
stop_word = stopwords.words('english') 
X_set = {i for i in X_list if not i in stop_word}

def cv(X_set, Y_set):
  r_vector = X_set.union(Y_set)
  list1 =[];list2 =[]
  for j in r_vector: 
      if j in X_set: list1.append(1)
      else: list1.append(0) 
      if j in Y_set: list2.append(1) 
      else: list2.append(0)
  return r_vector, list1, list2

def cc(r_vector, l1, l2):
  count = 0
  for i in range(len(r_vector)): 
        count+= list1[i]*list2[i] 
  c = count / float((sum(list1)*sum(list2))**0.5) 
  return c


list3 = []
for l in Data_Frame['After Spelling Correction']:
  Y_list = word_tokenize(l)
  Y_set = {k for k in Y_list if not k in stop_word}
  r_vector, list1, list2 = cv(X_set, Y_set)
  try:
    similarity = cc(r_vector, list1, list2)
  except ZeroDivisionError:
    similarity = 'None'
  list3.append(similarity)
Data_2 = pd.DataFrame(list(zip(Data_Frame['Reviews'],list3)), columns=['Reviews','cosine cimilarity'])
Data_2

,Reviews,cosine cimilarity
0,"Every once in a while a movie comes, that trul...",0
1,This is a movie that only those who have felt ...,0.097503
2,"Truly a masterpiece, The Best Hollywood film o...",0.0723102
3,Joaquin Phoenix gives a tour de force performa...,0.0432136
4,Most of the time movies are anticipated like t...,0.0218687
...,...,...
120,I didnt really know much about this movie. I h...,0.115517
121,,None
122,Film was directed and produced poorly. Opportu...,0.0672673
123,"Wow, what a movie! I have to admit, When I fir...",0.0792118


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:
# The GitHub link of your final csv file

# Link: https://github.com/mushamvineeth/Vineeth_INFO5731_Spring2021/blob/main/Reviews.numbers
